In [2]:
import pandas as pd
import numpy as np
import csv
import re
import pygsheets
import jieba
import jieba.analyse
import paddle
paddle.enable_static
import warnings
warnings.filterwarnings("ignore")
from gensim.models import word2vec

In [6]:
def read_sheet(sheet_url):
    # Authenticate with the service account credentials
    try:
        gc = pygsheets.authorize(client_secret='client_secret.json')
    except:
        print('get data from local')
        return list(pd.read_csv('nlp.csv', header=None)[0])

    # Open the public Google Sheet using its URL
    sheet = gc.open_by_url(sheet_url)

    # Select the first sheet (index 0)
    worksheet = sheet[0]

    # Get all values from the worksheet as a 2D list
    values = worksheet.get_col(1)
    non_empty_values = [value for value in values if value]

    return non_empty_values

data = read_sheet('https://docs.google.com/spreadsheets/d/1nqvhZIGhOaH93mAqMzcpuRwl-p1lbDw34hWbxuEYRho')


In [7]:
data[:3]

['玻尿酸是什麼', '玻尿酸是什麼做的？', '玻尿酸 對身體好嗎？']

# Jieba

In [8]:
jieba.enable_paddle()
jieba.set_dictionary('dict.txt.big')
jieba.add_word('玻尿酸')
textlist = [jieba.lcut(text, use_paddle=True) for text in data]
newtextlist = []
newtextlist = [[word for word in text if word != '' and word != ' '] for text in textlist]
newtextlist = [' '.join(i) for i in newtextlist]  

Import error, cannot find paddle.fluid and jieba.lac_small.predict module. Now, back to jieba basic cut......
[2023-07-18 01:24:27,919] [   DEBUG] _compat.py:50 - Import error, cannot find paddle.fluid and jieba.lac_small.predict module. Now, back to jieba basic cut......
Building prefix dict from /Users/lytt/Desktop/zeroone/NLP Clustering/dict.txt.big ...
[2023-07-18 01:24:27,921] [   DEBUG] __init__.py:113 - Building prefix dict from /Users/lytt/Desktop/zeroone/NLP Clustering/dict.txt.big ...
Loading model from cache /var/folders/kd/bdrqt3tj351gf36v20dm1wqr0000gn/T/jieba.u88881455df339fc89de811b30045d662.cache
[2023-07-18 01:24:27,922] [   DEBUG] __init__.py:132 - Loading model from cache /var/folders/kd/bdrqt3tj351gf36v20dm1wqr0000gn/T/jieba.u88881455df339fc89de811b30045d662.cache
Loading model cost 0.490 seconds.
[2023-07-18 01:24:28,411] [   DEBUG] __init__.py:164 - Loading model cost 0.490 seconds.
Prefix dict has been built successfully.
[2023-07-18 01:24:28,412] [   DEBUG] __in

In [9]:
newtextlist[:3]

['玻尿酸 是 什麼', '玻尿酸 是 什麼 做 的 ？', '玻尿酸 對 身體 好 嗎 ？']

# Word Embedding

## Gensim

In [10]:
# from gensim.corpora import WikiCorpus
# # Download from https://dumps.wikimedia.org/zhwiki/20230701/
# wiki_corpus = WikiCorpus('zhwiki-20230701-pages-articles.xml.bz2', dictionary={})
# text_num = 0

# with open('wiki_text.txt', 'w', encoding='utf-8') as f:
#     for text in wiki_corpus.get_texts():
#         f.write(' '.join(text)+'\n')
#         text_num += 1
#         if text_num % 10000 == 0:
#             print('{} articles processed.'.format(text_num))

#     print('{} articles processed.'.format(text_num))

In [11]:
# from opencc import OpenCC

# # Initial
# cc = OpenCC('s2t')

# # Tokenize
# with open('wiki_text_seg.txt', 'w', encoding='utf-8') as new_f:
#     with open('wiki_text.txt', 'r', encoding='utf-8') as f:
#         jieba.enable_paddle()
#         jieba.set_dictionary('dict.txt.big')
#         jieba.add_word('玻尿酸')
#         text_num=0
#         for times, data in enumerate(f, 1):
#             text_num += 1
#             if text_num % 10000 == 0:
#                 print('{} articles processed.'.format(text_num))
#             data = cc.convert(data)
#             data = jieba.lcut(data, use_paddle=True, cut_all=False)
#             data = [word for word in data if word != ' ']
#             data = ' '.join(data)

#             new_f.write(data)

In [12]:


# # Settings
# seed = 666
# sg = 0
# window_size = 10
# vector_size = 100
# min_count = 1
# workers = 8
# epochs = 5
# batch_words = 10000

# train_data = word2vec.LineSentence('wiki_text_seg.txt')
# model = word2vec.Word2Vec(
#     train_data,
#     min_count=min_count,
#     vector_size=vector_size,
#     workers=workers,
#     epochs=epochs,
#     window=window_size,
#     sg=sg,
#     seed=seed,
#     batch_words=batch_words
# )

# model.save('word2vec.model')

In [13]:
try:
    model = word2vec.Word2Vec.load('word2vec.model')
except FileNotFoundError:
    print('please uncomment the code above and train again')

In [14]:
from numpy import dot
from numpy.linalg import norm
from itertools import combinations

def cossim(a, b):
    cos_sim = dot(a, b)/(norm(a)*norm(b))
    return cos_sim

for a, b in combinations(['副作用', '好處', '功效', '後遺症'],2):
    v_a = model.wv[a]
    v_b = model.wv[b]
    print(a, b, ':', cossim(v_a, v_b))

副作用 好處 : 0.48933536
副作用 功效 : 0.653795
副作用 後遺症 : 0.68280816
好處 功效 : 0.525196
好處 後遺症 : 0.27764294
功效 後遺症 : 0.44948134


In [15]:
sent_vectors = []
for text in newtextlist:
    wordlist = text.split(' ')
    wordlist.remove('玻尿酸')
    sent_vector = np.zeros((model.vector_size,), dtype=np.float32)
    for word in wordlist:
        try:
            sent_vector+=model.wv[word]
        except:
            pass
    sent_vectors.append(list(sent_vector/len(wordlist)))
sent_vectors = np.array(sent_vectors)

In [16]:
sent_vectors.shape

(153, 100)

## TFIDF

In [17]:
from collections import Counter
from tqdm import tqdm
from scipy.sparse import csr_matrix
import math
import operator
from sklearn.preprocessing import normalize
import numpy

def IDF(corpus, unique_words):
    idf_vals = {}    
    total_docs = len(corpus) 
    for word in unique_words: 
        cnt = 0
        for row in corpus:
            if word in row.split(" "): 
                cnt+=1 
        idf_vals[word] = 1 + math.log((1+total_docs)/(1+cnt)) 
    return idf_vals

def fit(dataset):
    unique_words = set() 
    for row in dataset:
        for word in row.split(' '):
            unique_words.add(word)
    unique_words = sorted(list(unique_words))
    vocab = {j:i for i,j in enumerate(unique_words)}
    idfs_ = IDF(dataset, unique_words)
    return vocab, idfs_

def transform(dataset, features, idfs_):
    sparse_matrix = csr_matrix((len(dataset), len(features)), dtype=float)
    for row in range(0, len(dataset)):
        word_count = Counter(dataset[row].split(' '))
        for word in dataset[row].split(' '):
            if word in list(features.keys()):
                tf = word_count[word] / len(dataset[row].split(' '))
                tfidf = tf * idfs_[word]
                sparse_matrix[row, features[word]] = tfidf
    output = normalize(sparse_matrix, norm='l2', axis = 1, copy=True, return_norm=False)
    return output

def get_tfidf(textlist):
    features, idfs_ = fit(textlist)
    tfidf_matrix = transform(textlist, features, idfs_)
    feature_dict_custom = []
    tfidf = tfidf_matrix.toarray()
    for i in range(len(tfidf)):
        feature_dict_custom.append({key: value for key,value in zip(features.keys(), tfidf_matrix)})  
    return tfidf, feature_dict_custom

In [18]:
tfidf_jieba, feature_jieba = get_tfidf(newtextlist)

# Concatenate TFIDF and Average Word Embedding

In [19]:
all_feat = np.concatenate([sent_vectors, tfidf_jieba], axis=1)
all_feat.shape

(153, 154)


# Clustering

In [20]:
def cluster_result(labels):
    clusters = [[] for _ in range(NUM_CLUSTERS)]

    for idx, c in enumerate(labels):
        clusters[int(c)].append(data[idx])

    for c, result in enumerate(clusters):
        print('Cluster {}: {}'.format(c, ' '.join(result)))
        print()

## KMeans

In [21]:
from sklearn.cluster import KMeans
import numpy as np
NUM_CLUSTERS = 7

kmeans = KMeans(n_clusters=NUM_CLUSTERS, random_state=0, n_init="auto").fit(all_feat)
labels_kmeans = kmeans.labels_
cluster_result(labels_kmeans)

Cluster 0: 玻尿酸可以用喝的嗎？ 吃玻尿酸有什麼好處？ 玻尿酸用喝的有效嗎？ 玻尿酸用喝的有效嗎？ 玻尿酸可以用喝的嗎？ 玻尿酸吃的有效嗎？ 什麼食物有玻尿酸？ 什麼食物有玻尿酸？ 玻尿酸可以用喝的嗎？ 什麼食物有玻尿酸？ 什麼食物有玻尿酸？ 吃玻尿酸有什麼好處？ 玻尿酸用喝的有效嗎？ 玻尿酸可以喝嗎？ 玻尿酸可以用吃的嗎？

Cluster 1: 玻尿酸一天吃多少？ 玻尿酸一天吃多少？ 口服玻尿酸怎麼吃？ 玻尿酸一天吃多少？ 玻尿酸一天吃多少？ 玻尿酸一天吃多少？ 玻尿酸一天吃多少？ 玻尿酸一天吃多少？ 玻尿酸一天吃多少？ 玻尿酸一天吃多少？ 玻尿酸一天吃多少？ 玻尿酸一天吃多少？ 玻尿酸一天吃多少？ 口服玻尿酸怎麼吃？

Cluster 2: 玻尿酸 對身體好嗎？ 打玻尿酸會傷身體嗎？ 打玻尿酸會傷身體嗎？ 打玻尿酸會傷身體嗎？ 癌症可以打玻尿酸嗎？ 打玻尿酸多久會自然？ 玻尿酸多久可以打一次？ 打玻尿酸會傷身體嗎？ 玻尿酸怎麼補充？ 玻尿酸打完多久自然？ 打玻尿酸會傷身體嗎？ 膝蓋打玻尿酸後會痛嗎？ 膝蓋打完玻尿酸可以運動嗎？ 玻尿酸怎麼補充？ 打玻尿酸會傷身體嗎？ 玻尿酸打完多久自然？ 打玻尿酸會傷身體嗎？ 膝蓋打玻尿酸後會痛嗎？ 膝蓋打完玻尿酸可以運動嗎？ 玻尿酸怎麼補充？ 打玻尿酸會傷身體嗎？ 玻尿酸怎麼補充？ 癌症可以打玻尿酸嗎？ 打玻尿酸多久會自然？ 玻尿酸多久可以打一次？ 打玻尿酸會傷身體嗎？ 玻尿酸 對身體好嗎？ 癌症可以打玻尿酸嗎？ 打玻尿酸多久會自然？ 玻尿酸多久可以打一次？ 玻尿酸打完多久自然？ 打玻尿酸會傷身體嗎？ 打完玻尿酸幾天消腫？ 打完玻尿酸多久可以運動？

Cluster 3: 玻尿酸有什麼好處？ 玻尿酸有什麼副作用？ 玻尿酸有什麼副作用？ 玻尿酸有什麼好處？ 玻尿酸會致癌嗎？ 玻尿酸有什麼副作用？ 玻尿酸會有後遺症嗎？ 玻尿酸有什麼副作用？ 玻尿酸會致癌嗎？ 玻尿酸會有後遺症嗎？ 玻尿酸有什麼副作用？ 玻尿酸會致癌嗎？ 玻尿酸有什麼好處？ 玻尿酸有什麼副作用？ 玻尿酸有什麼好處？ 玻尿酸有什麼副作用？ 玻尿酸有什麼後遺症？ 玻尿酸有什麼副作用？ 玻尿酸會有後遺症嗎？ 玻尿酸有什麼副作用？ 玻尿酸有什麼好處？ 口服玻尿酸有什麼功效？ 口服玻尿酸有什麼功效？ 口服玻尿酸有什麼功效？ 玻尿酸會有後遺症嗎？ 口服玻尿酸有什麼功

## Hierarchical Clustering

In [22]:
from sklearn.cluster import AgglomerativeClustering

NUM_CLUSTERS = 7
hierarchical_cluster = AgglomerativeClustering(n_clusters=NUM_CLUSTERS, metric='euclidean', linkage='ward')
labels_hc = hierarchical_cluster.fit_predict(all_feat)
cluster_result(labels_hc)

Cluster 0: 玻尿酸可以用喝的嗎？ 玻尿酸什麼時候喝最好？ 玻尿酸什麼時候喝最好？ 玻尿酸什麼時候喝最好？ 玻尿酸用喝的有效嗎？ 玻尿酸什麼時候喝最好？ 玻尿酸什麼時候喝最好？ 玻尿酸用喝的有效嗎？ 玻尿酸可以用喝的嗎？ 玻尿酸什麼時候吃最好？ 玻尿酸吃的有效嗎？ 玻尿酸什麼時候喝最好？ 玻尿酸什麼時間吃最好？ 玻尿酸什麼時間吃最好？ 玻尿酸可以用喝的嗎？ 玻尿酸什麼時間吃最好？ 玻尿酸什麼時候喝最好？ 玻尿酸什麼時間吃最好？ 玻尿酸什麼時候喝最好？ 玻尿酸什麼時候喝最好？ 玻尿酸用喝的有效嗎？ 玻尿酸可以喝嗎？ 玻尿酸可以用吃的嗎？

Cluster 1: 玻尿酸 對身體好嗎？ 打玻尿酸會傷身體嗎？ 膝蓋打玻尿酸有什麼後遺症？ 打玻尿酸會傷身體嗎？ 膝蓋打玻尿酸有什麼後遺症？ 打玻尿酸會傷身體嗎？ 癌症可以打玻尿酸嗎？ 打玻尿酸多久會自然？ 玻尿酸多久可以打一次？ 膝蓋打玻尿酸有什麼後遺症？ 打玻尿酸會傷身體嗎？ 膝蓋打玻尿酸有什麼後遺症？ 打玻尿酸會傷身體嗎？ 膝蓋打玻尿酸有後遺症嗎？ 膝蓋打玻尿酸後會痛嗎？ 膝蓋打完玻尿酸可以運動嗎？ 膝蓋打玻尿酸有什麼後遺症？ 打玻尿酸會傷身體嗎？ 膝蓋打玻尿酸有什麼後遺症？ 打玻尿酸會傷身體嗎？ 膝蓋打玻尿酸有後遺症嗎？ 膝蓋打玻尿酸後會痛嗎？ 膝蓋打完玻尿酸可以運動嗎？ 膝蓋打玻尿酸有什麼後遺症？ 打玻尿酸會傷身體嗎？ 癌症可以打玻尿酸嗎？ 打玻尿酸多久會自然？ 玻尿酸多久可以打一次？ 膝蓋打玻尿酸有什麼後遺症？ 打玻尿酸會傷身體嗎？ 玻尿酸 對身體好嗎？ 癌症可以打玻尿酸嗎？ 打玻尿酸多久會自然？ 玻尿酸多久可以打一次？ 打玻尿酸會傷身體嗎？ 打完玻尿酸多久可以運動？

Cluster 2: 為何叫玻尿酸？ 口服玻尿酸怎麼吃？ 為何叫玻尿酸？ 玻尿酸又叫什麼？ 玻尿酸怎麼補充？ 玻尿酸打完多久自然？ 玻尿酸硬塊多久消？ 玻尿酸怎麼補充？ 玻尿酸何時喝？ 玻尿酸打完多久自然？ 玻尿酸硬塊多久消？ 玻尿酸怎麼補充？ 玻尿酸怎麼補充？ 口服玻尿酸怎麼吃？ 玻尿酸打完多久自然？ 玻尿酸硬塊多久消？ 打完玻尿酸幾天消腫？ 打玻尿酸會痛幾天？

Cluster 3: 玻尿酸是什麼 玻尿酸是什麼做的？ 玻尿酸是天然的嗎？ 玻尿酸是什麼成份做的？ 玻尿酸是天然的嗎？ 玻尿酸是尿嗎？ 玻尿酸是什麼成份做的？ 玻尿酸是天然的嗎？ 

# Upload to google sheet

In [23]:
try:
    gc = pygsheets.authorize(client_secret='client_secret.json')
except:
    print('no client_secret.json')

# Open the Google Sheet by title
sheet_title = 'NLP Clustering'
# Find the Google Sheets file by name
try:
    sheet = gc.open(sheet_title)
    print("Found the sheet:", sheet.title)
except pygsheets.SpreadsheetNotFound:
    print("The sheet with the name '{}' was not found. Creating a new one.".format(sheet_title))

    # Create a new spreadsheet with the given name
    sheet = gc.create(sheet_title)
    print("New sheet created with the title:", sheet.title)


worksheet = sheet[0]

Found the sheet: NLP Clustering


In [36]:
# Get the range where you want to write the data, for example, starting from A1
cell_range = 'A1:C' + str(len(data))

column1 = data
column2 = ['kmeans'] + [int(label) for label in labels_kmeans]
column3 = ['hierarchical clustering'] + [int(label) for label in labels_hc]

# Prepare the data as a list of lists to match the three columns
data_to_write = [column1, column2, column3]
data_to_write = [list(x) for x in zip(*data_to_write)]

# Write the data to the worksheet
worksheet.update_values(crange='A1', values=data_to_write)